In [ ]:
%load_ext autoreload
%autoreload 2

import functools
import os
import cv2
import matplotlib.pylab as plt
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import PIL
import sys
import glob
import subprocess
import pandas as pd
sys.path.append('../py')

import ai_utils
from image_utils import load_image, show_image, save_image
import image_utils as iu
import time

print("TF Version: ", tf.__version__)
print("TF Hub version: ", hub.__version__)
print("Eager mode enabled: ", tf.executing_eagerly())
print("GPU available: ", tf.config.list_physical_devices('GPU'))

In [ ]:
base_path = "/Users/iman/github/avignon/"    
project_dir =  "projects/graphite/"
input_dir = os.path.join(base_path, project_dir, "graphite_style")
mask_dir = os.path.join(base_path, project_dir, "graphite_original")
output_dir = os.path.join(base_path, project_dir, "output")
styles_dir = os.path.join(base_path, project_dir, "styles")
temp_dir = os.path.join(base_path, project_dir, "temp")

In [ ]:
input_files = glob.glob(os.path.join(mask_dir, '*'))
input_files.sort()

content_img_size = (1920, 1080)
style_img_size = (1920, 1080)

In [ ]:
def run_ebsynth(style, input_file, guide_1, out_dir, uniformity=1,patch_size=5, pyramid_levels=1, search_vote_iters=1, patch_match_iters=1, guide_1_weight=100, guide_2_weight=0.01, guide_2=None):
    output_file = os.path.splitext(os.path.basename(input_file))[0]
    if guide_2:
        command = f"ebsynth -style {style} \
                -guide {guide_1}  {input_file} -weight {guide_1_weight} \
                -guide {guide_2} {input_file} -weight {guide_2_weight} \
                -output {out_dir}/{output_file}.png \
                -uniformity {uniformity} -patchsize {patch_size} -pyramidlevels {pyramid_levels} -searchvoteiters {search_vote_iters} -patchmatchiters {patch_match_iters} "
    else:
        command = f"ebsynth -style {style} \
                -guide {guide_1}  {input_file} -weight {guide_1_weight} \
                -output {out_dir}/{output_file}.png \
                -uniformity {uniformity} -patchsize {patch_size} -pyramidlevels {pyramid_levels} -searchvoteiters {search_vote_iters} -patchmatchiters {patch_match_iters} "

    subprocess.call(command, shell=True)
    return f"{out_dir}/{output_file}.png"

In [ ]:
data = df = pd.read_csv(os.path.join(base_path, "questions/q_and_a_truth.csv"), sep="|",)
data["Numbers"]
colours = []
for number in data["Numbers"]:
    min_g_value = 1
    max_g_value = 124
    if '%' in number:
        number = float(number.replace('%', ''))
        colour = max_g_value*(number/100)
    elif 'jours' in number:
        colour = 122
    else:
        number = int(number)
        length = len(str(number))
        max_length = int('9' * length)
        colour = number/max_length*max_g_value

    colours.append(colour)


In [ ]:
80000/99999*123


In [ ]:
width = 1080
height = 1920
temp_files = []
for i in range(0,10):
    blank_image = np.zeros((height, width, 3), np.uint8)
    blank_image.fill(255)
    test_i = load_image(input_files[i])

    grid_rows = 8
    grid_cols = 4

    cell_width = test_i.shape[1] // 4
    cell_height = test_i.shape[0] // 8

    for row in range(grid_rows):
        for col in range(grid_cols):
            # Calculate the position of the cell
            x = col * cell_width
            y = row * cell_height

            # Place the text in the cell
            text = f'(80, 90)'
            cv2.putText(blank_image, text, (x+cell_width//2, y+cell_height//2), cv2.FONT_HERSHEY_SIMPLEX, 10, (255, 0, 0), 5)
    temp_file_path = os.path.join(temp_dir, os.path.basename(input_files[i]))
    # save_image(temp_file_path, test_i)
    temp_files.append(temp_file_path)
    s_image = iu.overlay(blank_image,test_i)
    s_image = iu.edges(s_image)
    s_image = cv2.cvtColor(s_image, cv2.COLOR_GRAY2BGR)
    s_image = iu.overlay(test_i, s_image, 0.5)
    s_image = iu.overlay(blank_image, s_image, 0.5)
    show_image(s_image, fig_size=(10,5))
    # break
    # save_image(temp_file_path, cv2.cvtColor(s_image, cv2.COLOR_BGR2GRAY))
    save_image(temp_file_path, s_image)


In [ ]:
output_frames = []
for i in range(0,10):
    input_file = temp_files[i]
    mask_file = os.path.join(mask_dir, os.path.basename(input_file))
    out_dir = output_dir
    # style = os.path.join(base_path,"projects/graphite/temp/0010.png")
    # style = os.path.join(base_path,"other/text_out2.jpg")
    style = os.path.join(temp_dir, "0000.png")

    # guide_2 = os.path.join(base_path,"other/text_out.jpg")
    guide_2 = None
    guide_1_weight = 1000
    guide_2_weight = 1
    start_time = time.time()
    output_frames.append(run_ebsynth(style=style, guide_2=guide_2, guide_1_weight=guide_1_weight, guide_2_weight=guide_2_weight, input_file=input_file, guide_1=mask_file, out_dir=out_dir))
    elapsed_time = time.time() - start_time
    print("Elapsed time: {:.2f} seconds".format(elapsed_time))

    show_image(load_image(output_frames[i]), fig_size=(2.5,5))


In [ ]:
# for i, x in enumerate(output_frames):
#     show_image(load_image(output_frames[i]), fig_size=(2.5,5))